In [1]:
from pyspark.sql import SparkSession

In [2]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [3]:
spark = SparkSession.builder.appName("xG6").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 02:34:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
events = spark.read.csv('Datas/events_shot.csv', header=True, inferSchema=True)

In [5]:
import pp_events as pp

In [6]:
train_data, test_data = pp.pre_training(events, features)

In [7]:
trained_model = pp.train_model(train_data,'gbt')

In [8]:
predictions = trained_model.transform(test_data)

In [9]:
from model_evaluation import ModelEvaluation

In [10]:
predictions = pp.goal_proba(predictions)

In [11]:
me = ModelEvaluation(predictions,'goal','prediction')

24/12/26 02:35:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [12]:
me_r = ModelEvaluation(predictions,'shot_statsbomb_xg','goal_probability',model_type='regression')

In [13]:
me_r.get_all_metrics()

{'MSE': 0.004101597637605669,
 'RMSE': 0.06404371661299545,
 'MAE': 0.043432260428111887,
 'R2': 0.8100443936383015}

In [14]:
me.get_all_metrics()

{'Accuracy': 0.9011106540518306,
 'Precision': 0.7598684210526315,
 'Recall': 0.1698529411764706,
 'Sensitivity': 0.1698529411764706,
 'Specificity': 0.9932376100046317,
 'F1': 0.2776442307692308,
 'FPR': 0.006762389995368226,
 'FNR': 0.8301470588235295}

In [19]:
spark.stop()